In [10]:
import pandas as pd
import json

In [ ]:
note = pd.read_csv('data/NOTEEVENTS.csv')

# Get admission IDs that have both admission notes and discharge summaries
admission_ids = note[note['DESCRIPTION'].str.contains('Admission Note', na=False)]['HADM_ID']
discharge_ids = note[note['CATEGORY'] == 'Discharge summary']['HADM_ID']
admissions_with_both = set(admission_ids) & set(discharge_ids)
filtered_notes = note[note['HADM_ID'].isin(admissions_with_both)]

# filtered_notes.to_csv('data/filtered_notes.csv', index=False)

In [11]:
filtered_notes = pd.read_csv('data/filtered_notes.csv')
filtered_notes['SUBJECT_ID'].unique()

array([26601, 96774, 62745, ..., 92606, 76012, 28475], shape=(5020,))

In [12]:
prescription = pd.read_csv('data/PRESCRIPTIONS.csv')
labs = pd.read_csv('data/labs_with_labels.csv')

/tmp/ipykernel_484594/3688311774.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescription = pd.read_csv('data/PRESCRIPTIONS.csv')


In [22]:
pt_id = 96774

In [23]:
note_type = ['Discharge summary', 'Physician ', 'Consult', 'Radiology']
progress_notes = filtered_notes[(filtered_notes['SUBJECT_ID'] == pt_id) & 
                                (filtered_notes['CATEGORY'].isin(note_type))]
# filter out duplicate CGID
progress_notes = progress_notes.drop_duplicates(subset=['CGID'])
# fitler out duplicate note time
progress_notes = progress_notes.drop_duplicates(subset=['CHARTTIME'])
progress_notes.to_csv(f'data/progress_notes_{pt_id}.csv', index=False)

In [24]:
sample_meds = prescription[prescription['SUBJECT_ID']==pt_id]
sample_meds.to_csv(f'data/sample_meds_{pt_id}.csv', index=False)

In [25]:
sample_lab = labs[(labs['SUBJECT_ID'] == pt_id) & (labs['FLAG'] == 'abnormal')]
sample_lab.to_csv(f'data/sample_lab_{pt_id}.csv', index=False)